<a href="https://colab.research.google.com/github/MohsinHoon/Liplingo-Final-Year-Project-2024-/blob/main/Final_Year_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python matplotlib imageio gdown tensorflow

In [ ]:
!pip install soundfile

In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio
import soundfile as sf

Configurring GPU for access

In [ ]:
tf.config.list_physical_devices('GPU')

Whether the GPU available on Not

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

**Building the Data Loading Function**

In [ ]:
import gdown

Fetching the dataset directly from the google drive available publcaly
Videos with their Alignments

In [ ]:
url = 'https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL'
output = 'data.zip'
gdown.download(url, output, quiet=False)
gdown.extractall('data.zip')

Loading the Video File and Pre Processing the annotations we will standanrized the video/images and making the cutout portion

In [ ]:
def load_video(path:str) -> List[float]:

    cap = cv2.VideoCapture(path)
    frames = []
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret, frame = cap.read()
        frame = tf.image.rgb_to_grayscale(frame)
        frames.append(frame[190:236,80:220,:])
    cap.release()

    mean = tf.math.reduce_mean(frames)
    std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    return tf.cast((frames - mean), tf.float32) / std

**Defining the Vocabulary, here we only define small alphabets as because we dont have any Capital form of data , but we still can add up on these**

In [ ]:
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]

Here, we are converting charchters to numbers and numbers to charchters and we are assinging one token to each charchter, i-e: Tokens are be like A=1 and Z=26

In [ ]:
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

Derving the Vocabulary

In [ ]:
char_to_num.get_vocabulary()

**Token can visible on this LOC**

In [ ]:
char_to_num(['l','i','p','l' ,'i', 'n', 'g', 'o' ])

**These show where the above charachter is on which number**

In [ ]:
num_to_char([12,9,16,12,9,14,7,15])

*Here, we are loading the Alingments for the video like where the as the sound and lip movements matches the Video, Thats why it is difficult to make a custom dataset because we are unable to figure the alingmnets* bold text

In [ ]:
def load_alignments(path:str) -> List[str]:
    with open(path, 'r') as f:
        lines = f.readlines()
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil':
            tokens = [*tokens,' ',line[2]]
    return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

**Testing the Path**

In [ ]:
test_path = '.\\data\\s1\\bbal6n.mpg'

**Through .tf we decoding the numpy array as we convert on above code**

In [ ]:
tf.convert_to_tensor(test_path).numpy().decode('utf-8').split('\\')[-1].split('.')[0]

'bbal6n'